In [1]:
import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

import pandas as pd
import json
import numpy as np
covid_file = '../data/COVID-QA.json'
jsonfile = open(covid_file, 'r')

covidQA = jsonfile.read()
jsonfile.close()

covid_data = json.loads(covidQA)
def convert_to_squad_format(data):
    covid_qa_squad_format = []
    for rows in data['data']:
      for context in rows['paragraphs']:
        for qa_pairs in context['qas']:
          features = {'id':str(context['document_id']),
                      'title': 'COVID_19',
                      'context':str(context['context']),
                      'question':qa_pairs['question'],
                      'answers':{'answer_start': np.array([qa_pairs['answers'][0]['answer_start']], dtype=np.int32),
                                 'text':[qa_pairs['answers'][0]['text']]}}
          covid_qa_squad_format.append(features)
    covid_df = pd.DataFrame(covid_qa_squad_format)
    return (datasets.Dataset.from_dict(covid_df))#.train_test_split(test_size=0.2)

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

Reusing dataset squad (/soe/meidam/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)


In [2]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val):
    !CUDA_VISIBLE_DEVICES=0 python run_qa.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/full_squad_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 1e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir


In [3]:
data_files = {}
data_files["train"] = covid_file

covid_qa = load_dataset('custom_squad.py', data_files=data_files)['train']
squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

Using custom data configuration default-7edf67a6e660d46e
Reusing dataset squad (/soe/meidam/.cache/huggingface/datasets/squad/default-7edf67a6e660d46e/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [ ]:
k_fold = 5
K = 5

for i in range(k_fold):
    covid_fold = covid_qa.shard(k_fold, i)
    
    covid_test = covid_fold.shard(2, 0)
    covid_val = covid_fold.shard(2, 1)    
    covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])   
    
    #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)
        
    checkpoint = 'roberta-base'
    cur_dir = '../models/gradual_ft_baseline_lr1e-5_2/split_' + str(i)
    
    squad_qa.shuffle()
    
    for n in range(K):
        output_dir = cur_dir + '/checkpoint_' + str(n)
        #squad_qa = datasets.Dataset.from_dict(squad_qa[:-to_remove_per_step])
        
        if n < K-1:
            squad_qa_cur = concatenate_datasets([squad_qa.shard(K, j) for j in range(K-n-1)])
            full_dataset = concatenate_datasets([squad_qa_cur, covid_train])
        elif n < K:
            squad_qa_cur = squad_qa.shard(K, 0)
            full_dataset = concatenate_datasets([squad_qa_cur, covid_train])
        else:
            full_dataset = covid_train
        
        make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)
        
        run_gradual_ft(output_dir, checkpoint, covid_val)
        
        checkpoint = output_dir
        



06/28/2021 09:53:44 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/28/2021 09:53:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=../models/gradual_ft_baseline_lr1e-5_2/split_0/checkpoint_0, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=32, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Jun28_09-53-44_nlp-gpu-01.soe.ucsc.edu, logging_strategy=IntervalStrategy.EPOCH, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.NO, save_steps=500, save_total_limi

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 60%|███████████████████████▏               | 2403/4037 [22:07<14:33,  1.87it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'loss': 0.8613, 'learning_rate': 0.0, 'epoch': 1.0}                            
100%|███████████████████████████████████████| 4037/4037 [37:04<00:00,  2.24it/s][INFO|trainer.py:1341] 2021-06-28 17:03:57,096 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 2224.1097, 'train_samples_per_second': 1.815, 'epoch': 1.0}   
100%|███████████████████████████████████████| 4037/4037 [37:04<00:00,  1.82it/s]
[INFO|trainer.py:1885] 2021-06-28 17:03:57,530 >> Saving model checkpoint to ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_0
[INFO|configuration_utils.py:351] 2021-06-28 17:03:57,531 >> Configuration saved in ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_0/config.json
[INFO|modeling_utils.py:889] 2021-06-28 17:03:58,681 >> Model weights saved in ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_0/pytorch_model.bin
[INFO|tokenization_utils_base.py:1924] 2021-06-28 17:03:58,682 >> tokenizer config file sav

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'loss': 0.2997, 'learning_rate': 0.0, 'epoch': 1.0}                            
100%|███████████████████████████████████████| 2174/2174 [17:05<00:00,  2.46it/s][INFO|trainer.py:1341] 2021-06-28 18:23:29,069 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 1025.7572, 'train_samples_per_second': 2.119, 'epoch': 1.0}   
100%|███████████████████████████████████████| 2174/2174 [17:05<00:00,  2.12it/s]
[INFO|trainer.py:1885] 2021-06-28 18:23:29,522 >> Saving model checkpoint to ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_3
[INFO|configuration_utils.py:351] 2021-06-28 18:23:29,523 >> Configuration saved in ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_3/config.json
[INFO|modeling_utils.py:889] 2021-06-28 18:23:30,536 >> Model weights saved in ../models/gradual_ft_baseline_lr1e-5_2/split_3/checkpoint_3/pytorch_model.bin
[INFO|tokenization_utils_base.py:1924] 2021-06-28 18:23:30,537 >> tokenizer config file sav